### Используемые индикаторы
* RSI
* Bollinger Bands
* On Balance Volume

## Описание стратегии
### Условия покупки
* Цена закрытия выше Rolling Mean
* RSI выше 50
* OBV > 0
### Условие продажи
* Stop loss, Take profit <= Bollinger Band Low


In [1]:
import warnings
import pandas as pd
import yfinance as yahooFinance
import plotly.graph_objects as go
from tqdm import tqdm
from plotly.subplots import make_subplots


warnings.filterwarnings("ignore")

In [2]:
def calculate_rsi(data, window):
    delta = data.diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    average_gain = up.rolling(window).mean()
    average_loss = abs(down.rolling(window).mean())
    rs = average_gain / average_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_bollinger_bands(data, window):
    rolling_mean = data.rolling(window).mean()
    rolling_std  = data.rolling(window).std()
    upper_band = rolling_mean + (rolling_std*2)
    lower_band = rolling_mean - (rolling_std*2)
    return rolling_mean, upper_band, lower_band

def calculate_obv(data: pd.DataFrame):
    obv = [0]
    for i in range(1, len(data.Close)):
        if data.Close[i] > data.Close[i-1]:
            obv.append(obv[-1] + data.Volume[i])
        elif data.Close[i] < data.Close[i-1]:
            obv.append(obv[-1] - data.Volume[i])
        else:
            obv.append(obv[-1])
    return obv

def prepare_data(df: pd.DataFrame, window=14) -> pd.DataFrame:
    df['RSI'] = calculate_rsi(df['Close'], window)
    df['Rolling Mean'], df['Bollinger High'], df['Bollinger Low'] = calculate_bollinger_bands(df['Close'], window)
    df['OBV'] = calculate_obv(df)
    return df

In [3]:
def strategy(df: pd.DataFrame):
    buy, sell = [], []
    last_signal = None
    for index, row in df.iterrows():
        if last_signal is None or last_signal == 'SELL':
            if row['RSI'] > 50 and row['Close'] > row['Rolling Mean'] and row['OBV'] > 0:
                buy.append(index)
                last_signal = 'BUY'
        else:
            if row['Close'] < row['Bollinger Low']:
                sell.append(index)
                last_signal = 'SELL'
    return buy, sell

In [4]:
df = yahooFinance.Ticker("AMZN").history(start="2020-01-01", end="2022-01-01")

In [5]:
df = prepare_data(df)

In [6]:
buy, sell = strategy(df)

In [8]:
fig = make_subplots(rows=3, cols=1, row_heights=[0.7, 0.15, 0.15])

fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             close=df['Close'],
                             high=df['High'],
                             low=df['Low'], name='Market'))

fig.add_trace(go.Scatter(x=df.index, y=df['Rolling Mean'], name='RM', line={'color': 'rgb(0,0,255)'}))
fig.add_trace(go.Scatter(x=df.index, y=df['Bollinger High'], name='Bollinger High', line={'color': 'rgb(128,128,128)'}))
fig.add_trace(go.Scatter(x=df.index, y=df['Bollinger Low'], name='Bollinger Low', line={'color': 'rgb(128,128,128)'}))

fig.add_trace(go.Scatter(
    x=buy, 
    y=[df.loc[x]['Close'] for x in buy],
    mode='markers',
    marker=dict(
        symbol='triangle-up',  # Устанавливаем форму маркеров в виде треугольников
        size=20,
        color='Green'
    ),
    name='Buy Signal'
))

fig.add_trace(go.Scatter(
    x=sell, 
    y=[df.loc[x]['Close'] for x in sell],
    mode='markers',
    marker=dict(
        symbol='triangle-down',  # Устанавливаем форму маркеров в виде треугольников
        size=20,
        color='Red'
    ),
    name='Sell Signal'
))

fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(
    title='Стратегия',
    yaxis_title='Цена акции',
    height=720,
    width=1500
)

fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], name='RSI'), row=2, col=1)
fig.add_hline(y=50, row=2, col=1, line_dash="dash")

fig.add_trace(go.Scatter(x=df.index, y=df['OBV'], name='OBV'), row=3, col=1)

fig.show()

## Тестирование на S&P500 2020-01-01 по 2022-01-01

In [ ]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']

In [195]:
profits = {}
for ticker in tqdm(sp500):
    df = yahooFinance.Ticker(ticker).history(start="2020-01-01", end="2022-01-01")
    df = prepare_data(df)
    buy, sell = strategy(df)
    
    buy_price = [df.loc[x]['Close'] for x in buy]
    sell_price = [df.loc[x]['Close'] for x in sell]
    
    profit = 0
    for idx in range(len(sell_price)):
        profit += sell_price[idx] - buy_price[idx]
    
    profits |= {ticker: profit}

100%|██████████| 503/503 [01:21<00:00,  6.18it/s]


In [196]:
result = pd.DataFrame(sorted(profits.items(), key=lambda x: x[1], reverse=True), columns=['Ticker', 'Profit'])

In [197]:
result

,Ticker,Profit
0,AZO,754.859985
1,CMG,664.959839
2,MTD,590.850220
3,BLK,334.023804
4,IDXX,313.399994
...,...,...
498,BIIB,-132.110016
499,FLT,-136.040009
500,LMT,-147.721069
501,EQIX,-196.959747


In [200]:
result['Profit'].sum()

9706.435075998306

## Тестирование на S&P500 2022-01-01 по текущий день

In [201]:
profits = {}
for ticker in tqdm(sp500):
    df = yahooFinance.Ticker(ticker).history(period='2y')
    df = prepare_data(df)
    buy, sell = strategy(df)
    
    buy_price = [df.loc[x]['Close'] for x in buy]
    sell_price = [df.loc[x]['Close'] for x in sell]
    
    profit = 0
    for idx in range(len(sell_price)):
        profit += sell_price[idx] - buy_price[idx]
    
    profits |= {ticker: profit}

100%|██████████| 503/503 [01:23<00:00,  6.04it/s]


In [202]:
result = pd.DataFrame(sorted(profits.items(), key=lambda x: x[1], reverse=True), columns=['Ticker', 'Profit'])

In [203]:
result

,Ticker,Profit
0,FICO,268.309998
1,GWW,262.518097
2,TDG,217.306732
3,NVDA,208.492157
4,MCK,126.703354
...,...,...
498,MPWR,-202.967163
499,MTD,-243.720093
500,BKNG,-291.559814
501,CMG,-352.680054


In [204]:
result['Profit'].sum()

-9205.313833236694

Данная стратегия показала себя положительно на отрезке с 2020 по 2022 год, но если брать последние два года, то стратегия демонстрирует плохой результат. Если учитывать ситуацию в мире и на рынке в с 2022 по 2023 год, то можно сказать, что данная стратегия не справляется в случае, когда внешние факторы сильно влияют на поведение рынка